In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(10)
tf.config.threading.set_intra_op_parallelism_threads(10)

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=6000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

import time

from vpe.utilities import tf, tfp, np, plt, tfb, floatf, tffloat, tfint, updateTfFloat,  _0
updateTfFloat ( tf.float32 )

from vpe.poisson1DPDEState import PDE_State
from vpe.poisson1DSolver import Solver 
from vpe.NeuralNet import NeuralN
from vpe.VEModel import VE
tf.random.set_seed(42)

import time

print(tffloat(0.))
tf.config.list_physical_devices()

dirname = 'weightsModel-Test1/'
# dirname = 'indptPrior-Test1/'

os.makedirs(os.path.dirname(dirname), exist_ok=True)


In [ ]:
'''Create PDEState Object with Cheby Num terms (-1 for degree)'''
omegaLength = 2.0
numFElems   = 100
numFENodes  = numFElems + 1
xFE = tffloat( tf.linspace( 0., omegaLength, numFElems + 1) ) - 1.

pdeState = PDE_State(xFE, dimK=4, dimF=1, dimU=8)
'''Modify pdeState Bijcetor for kappa'''
pdeState.kappaBij   = tfb.Chain( bijectors=[tfb.Shift(tffloat(1.)), tfb.Softplus()] )


In [ ]:
'''Create solver Object'''

solver = Solver( pdeState, xFE, isNonLinear=False )


# '''Modify pdeState.stress-strain for nonlinear Youngs Modulus'''
# import types
# def stress_strain_nonLinear(self, index, kappa, u_trial):
#     #print('custom stress-strain')
#     dudx = ( u_trial[index] - u_trial[index+1]) / (self.pdeState.xFE[index] - self.pdeState.xFE[index+1])
#     return (tf.keras.activations.sigmoid(tf.math.abs(dudx)) + kappa*5. ) /10.
# solver.stress_strain = types.MethodType(stress_strain_nonLinear, solver)


In [ ]:
model = VE( pdeState, solver )
#model.dimU = numFENodes
#model.logProbPhysResidual = model.physicsPenalty

def createNNsPrior():
    dimKFBlBr = model.dimK + model.dimF + model.dimBl + model.dimBr
    nh = 50
    U_KFBlBr  = NeuralN('U_of_KFBlBr', 
                    np.array([dimKFBlBr,
                              nh, nh, nh, nh,
                              model.dimU])
                    )

    k  = model.dimK
    bl = k  + model.dimBl
    br = bl + model.dimBr
    Z_UF     = NeuralN('Z_of_UF',
                    np.array([pdeState.dimU + model.dimF, 
                                nh, nh, nh, nh, 
                                model.dimK + model.dimBl + model.dimBr]),
                    multOutputLayout = {'K': tf.range(0,k),
                                        'Bl':tf.range(k,bl),'Br':tf.range(bl,br)}
                    )

    model.initPriorNetworks(U_KFBlBr=U_KFBlBr, Z_UF=Z_UF)
    model.setModePrior()
    return 

    
USE_UNIFORM_PRIOR = False
if USE_UNIFORM_PRIOR == True:
    model.LatentKappaPrior   = tfp.distributions.Uniform(low=tffloat([-0.5]*pdeState.dimK), high=tffloat([0.5]*pdeState.dimK))#works
    model.LatentForcingPrior = tfp.distributions.Uniform(low=tffloat([5.]*pdeState.dimF), high=tffloat([5.]*pdeState.dimF) )

USE_NORMAL_PRIOR = True
if USE_NORMAL_PRIOR == True:
    model.LatentKappaPrior   = tfp.distributions.Uniform(low=tffloat([-1.]*pdeState.dimK), high=tffloat([1.]*pdeState.dimK) )
    model.LatentForcingPrior = tfp.distributions.Uniform(low=tffloat([1.]*pdeState.dimF), high=tffloat([5.]*pdeState.dimF) )#works


model.LatentBlPrior      = tfp.distributions.Uniform(low=tffloat([-0.5]), high=tffloat([0.5]))
model.LatentBrPrior      = tfp.distributions.Uniform(low=tffloat([-0.5]), high=tffloat([0.5]) )


print(model.LatentForcingPrior.sample())
print(model.LatentBlPrior.sample())
createNNsPrior()


In [ ]:
totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

In [ ]:
totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

def computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=True, SAVE=False):
    
    global totalTimeNNSolving
    global totalTimeSolving  
    
    startSolving = time.time()
    meanSamplesU, logVarSamplesU = model.priorModels['U_of_KFBlBr'].Map([samplesK, samplesF, samplesBcl, samplesBcr])
    endSolving = time.time()
    totalTimeNNSolving += endSolving - startSolving

    us = model.reparameterize(meanSamplesU, logVarSamplesU )

    residualsU = tf.constant(0., shape=[1])
    residualsK = tf.constant(0., shape=[1])
    residualsB = tf.constant(0., shape=[1])
    
    if PLOT == True:
        fig, ax = plt.subplots(nrows=1, ncols=5, sharex=True,
                                            figsize=(24, 4))
    
    for i in range(samplesK.shape[0]):

        pdeState.kappa   = samplesK[i,:] 
        pdeState.forcing = samplesF[i,:] 
        pdeState.bdLeft  = samplesBcl[i,0] 
        pdeState.bdRight = samplesBcr[i,0] 

        startSolving = time.time()
        u_true     = solver.solve_tf_linear(  )
#         u_true     = solver.solve_tf_bfgs(  )
#         u_trueSpec = solver.solve_tf_FESpectral_bfgs( )
#         u_trueSpec = pdeState.u_funXFE( u_trueSpec[:,None] )
        endSolving = time.time()
        totalTimeSolving += endSolving - startSolving

        meanU, diagVarU = pdeState.gaussChebyGaussCoeffs(meanSamplesU[i,:], tf.exp(logVarSamplesU[i,:]), xFE )
        #meanU, diagVarU = meanSamplesU[i,:], tf.exp(logVarSamplesU[i,:])
        std2U = 2.*tf.sqrt(diagVarU)
        
        startSolving = time.time()
        uField = pdeState.u_funXFE(us[i,:][:,None])
        #uField  = us[i,:]
        endSolving = time.time()
        totalTimeNNSolving += endSolving - startSolving

        kappaField = pdeState.kappa_fun( xFE )

        U_true_coeffs = pdeState.getChebyCoeffs_fromPoints(xFE, uField, model.dimU - 1)
        #U_true_coeffs = uField
        #print('U_true_coeffs = ', U_true_coeffs)


        ZCoeffReconstMean, ZCoeffReconstlogvar = model.priorModels['Z_of_UF'].Map([ U_true_coeffs[None,:], samplesF[i,:][None,:]])
        kappaCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstMean =  tf.reshape(kappaCoeffReconstMean,[-1])
        kappaCoeffReconstVar  =  tf.exp(tf.reshape(kappaCoeffReconstlogVar,[-1]))
        meanK, diagCovK = pdeState.gaussChebyGaussCoeffs(kappaCoeffReconstMean,kappaCoeffReconstVar, xFE)
        meanKT, varKT = pdeState.unscentedGaussTransf(meanK, diagCovK, pdeState.kappaBij, k=2)
        std2KT = 2*tf.sqrt(varKT)
        print('std2KT = ', std2KT)

        BlReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Bl'], axis=1)
        BrReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Br'], axis=1)

        residualUi = ( tf.linalg.norm(  u_true - meanU ) / tf.linalg.norm( u_true ) )**2.
        residualKi = ( tf.linalg.norm( kappaField - meanKT ) / tf.linalg.norm( kappaField ) )**2.
        residualBi = ( tf.linalg.norm( ( BlReconst - samplesBcl[i,0] ) + ( samplesBcr[i,0] - BrReconst ) ) / tf.linalg.norm( samplesBcl[i,0] + samplesBcr[i,0] ) )**2.

        residualsU = tf.concat([residualsU, residualUi[None]], 0)
        residualsK = tf.concat([residualsK, residualKi[None]], 0)
        residualsB = tf.concat([residualsB, residualBi[None]], 0)

        
        if PLOT == True:
            #ax[0].plot( xFE, u_trueSpec, '--x', c='k', alpha = 0.1)
            ax[0].plot( xFE , u_true)
            ax[0].set_xlabel(r'$x$')
            ax[0].set_ylabel(r'$u(x)$')
            ax[0].set_title('FE Solver')
            
            ax[1].plot(xFE, meanU)
            ax[1].fill_between(xFE, meanU + std2U, meanU - std2U, alpha=0.2)
            ax[1].set_xlabel(r'$x$')
            ax[1].set_ylabel(r'$u(x) \pm 2\sigma$')
            ax[1].set_title('NN Solver')

            ax[2].plot( xFE, kappaField)
            ax[2].set_xlabel(r'$x$')
            ax[2].set_ylabel(r'$\kappa(x)$')
            ax[2].set_title(r'True $\kappa(x)$')
            
            ax[3].plot(xFE, meanKT)
            ax[3].fill_between(xFE, meanKT + std2KT, meanKT- std2KT, alpha=0.2)
            ax[3].set_xlabel(r'$x$')
            ax[3].set_ylabel(r'$\kappa(x) \pm 2\sigma$')
            ax[3].set_title(r'Reconstructed $\kappa(x)$')

            ax[4].plot( xFE , tf.squeeze(pdeState.forcing_funXFE()))
            ax[4].set_xlabel(r'$x$')
            ax[4].set_ylabel(r'$f(x)$')
            ax[4].set_title(r'Forcing')


    if PLOT == True:
        plt.tight_layout()
        if SAVE == True:
            plt.savefig('figs/OneDpoissonNONLinearInRangeSolves.pdf')
        plt.show()

    return residualsU, residualsK, residualsB

In [ ]:
numSample = 3
PLOT = True
sampleKGenerator = model.LatentKappaPrior
samplesK   = sampleKGenerator.sample(sample_shape=(numSample))

samplesF   = model.LatentForcingPrior.sample(sample_shape=(numSample))
samplesBcl = model.LatentBlPrior.sample(sample_shape=(numSample))
samplesBcr = model.LatentBrPrior.sample(sample_shape=(numSample))

totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

residualsU, residualsK, residualsB = computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=PLOT, SAVE=False)

print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
eps =1e-2

AllEpsU = []
AllEpsK = []
AllEpsB = []

totalTimeTraining  = 0.

trainModels      = False


intervalELBOsave = 1000

PLOT = True


@tf.function
def train_step_prior(optimizer, iteration):
    with tf.GradientTape(persistent=True) as tape:
        loss = model.compute_nelbo_prior(iteration)
    for priorvars in model.priorModels['trainable_variables']:
        gradients    = tape.gradient(loss, priorvars)
        gradients, globNorm = tf.clip_by_global_norm(gradients, 1.0)
        tf.debugging.check_numerics(gradients[0], "GRAD NOT FINITE")
        #gradients = [tf.where(tf.is_nan(grad), tf.zeros_like(grad), grad) for grad in gradients]
        optimizer.apply_gradients(zip(gradients, priorvars))
    return loss

if trainModels:
       
    '''Optimization sequence'''
    model.n_sgd      = 1

    num_iterations   = tfint(1_000_000)

    model.epsilon_r.assign( tffloat(eps) )

    lr = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-3, int(num_iterations/5), 0.5, staircase=True, name=None
    )
    optimizer = tf.keras.optimizers.Adam( lr )

    elboAll     = []
    startTraining = time.time()
    for epoch in tf.range(0, num_iterations ):

        start_time = time.time()
        loss = train_step_prior(optimizer, epoch)

        end_time = time.time()

        if epoch % intervalELBOsave == 0:
            #tf.print('epsi_r = ', model.epsilon_r)
            elboAll.append(-loss)
            print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
                .format(epoch, -loss, end_time - start_time))

    endTraining = time.time()
    totalTimeTraining += endTraining - startTraining
    print("DONE TRAINING FOR EPSI", eps)




In [ ]:
'''If you want to load Model Weights'''
saveModelWeights = False
loadModelWeights = True
if saveModelWeights:
    print('MODEL WEIGHTS SAVED')
    
    os.makedirs(os.path.dirname(dirname), exist_ok=True)
    model.priorModels['U_of_KFBlBr'].NN.save_weights(dirname+'U_of_KFBlBr_weights.h5')
    model.priorModels['Z_of_UF'].NN.save_weights(dirname+'Z_of_UF_weights.h5')
    
    model.priorModels['U_of_KFBlBr'].NN.compile() 
    model.priorModels['Z_of_UF'].NN.compile()
    model.priorModels['U_of_KFBlBr'].NN.save(dirname+'U_of_KFBlBr_weights.model')
    model.priorModels['Z_of_UF'].NN.save(dirname+'Z_of_UF_weights.model')

if loadModelWeights:
    print('LOADED WEIGHTS SAVED')

    model.priorModels['U_of_KFBlBr'].NN = tf.keras.models.load_model(dirname+'U_of_KFBlBr_weights.model')
    model.priorModels['Z_of_UF'].NN = tf.keras.models.load_model(dirname+'Z_of_UF_weights.model')

    model.priorModels['U_of_KFBlBr'].NN.compile()
    model.priorModels['Z_of_UF'].NN.compile()
    # Check its architecture
    model.priorModels['U_of_KFBlBr'].NN.summary()
    model.priorModels['Z_of_UF'].NN.summary()


#print(elboAll)
if trainModels:
    elboAll =  np.array(elboAll)
    np.savetxt('ELBOValsModel', elboAll)



In [ ]:
print( 'totalTimeTraining = ', totalTimeTraining)
print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
numSample  = 5
samplesK   = model.LatentKappaPrior.sample(sample_shape=(numSample))
samplesF   = model.LatentForcingPrior.sample(sample_shape=(numSample))
samplesBcl = model.LatentBlPrior.sample(sample_shape=(numSample))
samplesBcr = model.LatentBrPrior.sample(sample_shape=(numSample))

totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

residualsU, residualsK, residualsB = computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=PLOT, SAVE=False)

print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
plt.plot(range(elboAll.shape[0]),  elboAll )
scale = 1e3
plt.ylim(-scale,scale)
plt.show()

In [ ]:
with open('Times.txt', 'w') as the_file:
    the_file.write('totalTimeTraining = '  + str(totalTimeTraining) + 's'+ '\n' )
    the_file.write('totalTimeSolving = '  + str(totalTimeSolving) + 's'+  '\n' )
    the_file.write('totalTimeNNSolving = ' + str(totalTimeNNSolving) + 's'+  '\n' )

In [ ]:
model.numEachSide = 40
dataYEpsi = tffloat(0.010)
tfNan     = tf.constant(np.nan)

# def observationMap( u, x ):
#     u = tf.reshape(u, [-1])
#     x = tf.reshape(x, [-1])
#     return tf.sin( x * 20.) * u     , x
#     return tf.sin( x * 20 ) / 20 + u, x
#     return u, x

def observationMap(u, x):

    u = tf.reshape(u, [-1])
    x = tf.reshape(x, [-1])

    x = tf.concat([x[:model.numEachSide],
        x[-model.numEachSide:]], 0)

    u = tf.concat([u[:model.numEachSide],
        u[-model.numEachSide:]], 0)

    return u, x 

def addIndtNoise(u):
    return u + tf.random.normal(stddev=dataYEpsi, shape=u.shape, dtype=floatf)

def plotTrueSlnSamples(samplesK, samplesF, samplesBcl, samplesBcr):
    
    fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=4, sharex=False,
                                        figsize=(20, 3))
    
    u_All = tf.constant(0., shape=[1,  int(2*model.numEachSide)], dtype=floatf)
#     u_All = tf.constant(0., shape=[1,  101], dtype=floatf)

    #uInit = solver.solve_tf(  )
    #A, f  = solver.assemble_tf( uInit )
    #precond = tf.linalg.inv(A)

    for i in range(samplesK.shape[0]):
        
        model.pdeState.forcing = tf.reshape(samplesF[i,:], [-1]) 
        model.pdeState.kappa   = tf.reshape(samplesK[i,:], [-1]) 
        model.pdeState.bdLeft  = tf.reshape(samplesBcl[i,:], []) 
        model.pdeState.bdRight = tf.reshape(samplesBcr[i,:], []) 

#         u_true       = solver.solve_tf_bfgs(  )
        u_true       = solver.solve_tf_linear( )
        # print('u_true.shape = ', u_true.shape)
        u_obs, x_obs = observationMap(u_true, xFE)
        # print('u_obs.shape = ', u_obs.shape)
        u_obs_noise  = addIndtNoise(u_obs)
        # print('u_obs_noise.shape = ', u_obs_noise.shape)
        u_obs_noise_dropNan = u_obs_noise # tf.gather_nd(u_obs_noise, tf.where(tf.math.is_finite(u_obs_noise)))
#         u_obs_noise_dropNan = tf.gather_nd(u_obs_noise, tf.where(tf.math.is_finite(u_obs_noise)))
        # print('u_obs_noise_dropNan.shape = ', u_obs_noise_dropNan.shape)
        u_All = tf.concat([u_All,u_obs_noise_dropNan[None, :] ], 0)

#         x_obsp = x_obs # tf.concat([x_obs[:model.numEachSide], tf.constant([np.nan]),x_obs[-model.numEachSide:]], 0)
#         u_obsp = u_obs # tf.concat([u_obs[:model.numEachSide], tf.constant([np.nan]),u_obs[-model.numEachSide:]], 0)
#         u_obs_noisep = u_obs_noise # tf.concat([u_obs_noise[:model.numEachSide], tf.constant([np.nan]),u_obs_noise[-model.numEachSide:]], 0)
        x_obsp = tf.concat([x_obs[:model.numEachSide], tf.constant([np.nan]),x_obs[-model.numEachSide:]], 0)    
        u_obsp =  tf.concat([u_obs[:model.numEachSide], tf.constant([np.nan]),u_obs[-model.numEachSide:]], 0)
        u_obs_noisep =  tf.concat([u_obs_noise[:model.numEachSide], tf.constant([np.nan]),u_obs_noise[-model.numEachSide:]], 0)

        
        ax0.plot(x_obsp, u_obsp)
        # ax0.set_title('u_mapped')
        ax1.plot(x_obsp,u_obs_noisep)
        # ax1.set_title('u_mappedNoise')
        ax2.plot(tf.range(u_true.shape[0]), u_true)
        # ax2.set_title('u')

        kappaField = pdeState.kappa_fun(xFE)
        ax3.plot( xFE, kappaField )
        # ax3.set_title('kappa field')
    fig.canvas.draw()
    extent = ax0.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('u_obs.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    extent = ax1.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('u_obsNoise.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    extent = ax2.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('u_true.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    extent = ax3.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('TrueKappa.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    plt.show()
    u_All = u_All[1:,:]
    return u_All

model.dimY = pdeState.xFE.shape[0]

numData = 100
print('model.dimK = ', model.dimK)
samplesKdata   = model.LatentKappaPrior.sample(sample_shape=(numData))
samplesFdata   = model.LatentForcingPrior.sample(sample_shape=(numData))
samplesBldata  = model.LatentBlPrior.sample(sample_shape=(numData))
samplesBrdata  = model.LatentBrPrior.sample(sample_shape=(numData))


priorFEpsi = tf.constant(1e-3,dtype=floatf)
priorMeanF = samplesFdata #+ tf.random.normal(stddev=priorFEpsi,  shape=samplesFdata.shape, dtype=floatf)
# priorMeanF = samplesFdata * 0  + 2.5
priorFEpsi = tffloat(1.)
priorStdF  = tf.tile(priorFEpsi[None,None], priorMeanF.shape)
model.dataForcingPrior = (priorMeanF, tf.math.log(priorStdF)*2 )

variableSTD = tffloat(1e-1)

priorKEpsi = variableSTD
priorMeanK = samplesKdata #+ tf.random.normal(stddev=priorKEpsi,  shape=samplesKdata.shape, dtype=floatf)
# priorMeanK = samplesKdata * 0 
priorKEpsi = tffloat(0.5)
priorStdK  = tf.tile(priorKEpsi[None,None], priorMeanK.shape)
model.dataKappaPrior = (priorMeanK,  tf.math.log(priorStdK)*2)

priorBlEpsi = variableSTD
priorMeanBl = samplesBldata #+ tf.random.normal(stddev=priorBlEpsi,  shape=samplesBldata.shape, dtype=floatf)
# priorMeanBl = priorMeanBl * 0 
priorBlEpsi = tffloat(0.5)
priorStdBl  = tf.tile(priorBlEpsi[None,None], priorMeanBl.shape)
model.dataBlPrior = (priorMeanBl,  tf.math.log(priorStdBl)*2)

priorBrEpsi = variableSTD
priorMeanBr = samplesBrdata #+ tf.random.normal(stddev=priorBrEpsi,  shape=samplesBrdata.shape, dtype=floatf)
# priorMeanBr = priorMeanBr * 0 
priorBrEpsi = tffloat(0.5)
priorStdBr  = tf.tile(priorBrEpsi[None,None], priorMeanBr.shape)
model.dataBrPrior = (priorMeanBr,  tf.math.log(priorStdBr)*2)

model.dataY = plotTrueSlnSamples(samplesKdata, samplesFdata, samplesBldata, samplesBrdata)
print(model.dataY)



In [ ]:
np.savetxt("{}ModelDataset".format(dirname), model.dataY)

In [ ]:
print(model.dataForcingPrior)

In [ ]:
model.dimY = int( model.numEachSide * 2 )
# model.dimY = 101


def createNNsPost():
   
    nh = 100
    U_Y = NeuralN('U_of_Y',
                    np.array([model.dimY,
                      nh, nh, 
                      model.dimU]))

    model.U_Y      = U_Y
    model.U_Y.compile()

    model.varListPostNets = [ 
                               model.U_Y.trainable_variables
                            ]
    
    model.postModels = {
                         model.U_Y.NN_name:model.U_Y,
                         'trainable_variables':model.varListPostNets
                        }

    return 

createNNsPost()

In [ ]:
def compute_nelbo_post(self, iteration):

    nelboSGD = 0.
    i = iteration % self.dataY.shape[0] 

    f_i  =  self.dataForcingPrior[0][i,:][None, :]
    print('f_i = ', f_i)

    mean_phi, logvar_phi = self.postModels['U_of_Y'].Map( [self.dataY[i,:][None,:]] )
    us                   = self.reparameterize( mean_phi, logvar_phi )
    logq_phi_u_y         = self.log_normal_pdf( us, mean_phi, logvar_phi )

    uFE      = tf.reshape(self.pdeState.u_funXFE(tf.reshape(us, [-1,1])) , [1,-1] )

    g_u, x   = observationMap( uFE, pdeState.xFE )

    logP_y_u = self.log_normal_pdf(self.dataY[i,:][None,:], g_u[None,:], 2.*tf.math.log(self.epsilon_y) )

    elbo      = tf.reduce_mean(  logP_y_u  - logq_phi_u_y   )

    nelboSGD -= elbo

    return nelboSGD  

import types
model.compute_nelbo_post = types.MethodType(compute_nelbo_post, model)
 

In [ ]:

totalTimeTrainingPost  = 0.

trainModels      = True


intervalELBOsave = 1000

PLOT = True

model.epsilon_y.assign(tffloat(0.01)) #res1
# model.epsilon_r.assign(tffloat(0.1))
# model.epsilon_y.assign(dataYEpsi)

# model.setModePost()
# model.postModels['trainable_variables'] = [model.postModels['KBlBr_of_YF'].trainable_variables]

# model.setModeCouple()

loss = model.compute_nelbo_post(0)
@tf.function
def train_step_post(optimizer, iteration):
    with tf.GradientTape(persistent=True) as tape:
        loss = model.compute_nelbo_post(iteration)
        print('loss  = ', loss)

    for postVars in [ model.postModels['U_of_Y'].trainable_variables ]:
        gradients = tape.gradient(loss, postVars)
        optimizer.apply_gradients(zip(gradients, postVars))
    
#     for priorVars in [model.priorModels['Z_of_UF'].trainable_variables]:
#         gradients = tape.gradient(loss, priorVars)
#         optimizer.apply_gradients(zip(gradients, priorVars))
        
    return loss


if trainModels:
    
    createNNsPost()
    
    '''Optimization sequence'''
    model.n_sgd      = 1

    num_iterations   = tfint(500_000)

    # model.epsilon_r.assign( tffloat(eps) )

    lr = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4, int(num_iterations/10), 0.5, staircase=True, name=None
    )
    optimizer = tf.keras.optimizers.Adam( lr )
    # optimizer = tf.keras.optimizers.Adam( 1e-3 )


    elboAll       = []
    startTraining = time.time()
    for epoch in tf.range(0, num_iterations ):

        start_time = time.time()
        loss = train_step_post(optimizer, epoch)

        end_time = time.time()

        if epoch % intervalELBOsave == 0:
            #tf.print('epsi_r = ', model.epsilon_r)
            elboAll.append(-loss)
            print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
                .format(epoch, -loss, end_time - start_time))

    endTraining = time.time()
    totalTimeTraining += endTraining - startTraining
    print("DONE TRAINING FOR EPSI", eps)



In [ ]:
elboNP = np.array(elboAll)
plt.plot(range(elboNP.shape[0]), elboAll)
# plt.ylim(-1e6,1e6)
plt.show()

In [ ]:
saveModelWeights = False
loadModelWeights = True
'''If you want to load Model Weights'''

if loadModelWeights:
#     model.postModels['KBlBr_of_Y'].NN.load_weights(dirname+'/KBlBr_of_Y_weights.h5')
    model.postModels['U_of_Y'].NN.load_weights(dirname+'/U_of_Y_weights.h5')

#     model.postModels['KBlBr_of_Y'].NN.compile()
    model.postModels['U_of_Y'].NN.compile()
    # Check its architecture
#     model.postModels['KBlBr_of_Y'].NN.summary()
    model.postModels['U_of_Y'].NN.summary()
    print("LOADED WEIGHTS")




if saveModelWeights:
    # dirname = 'weightsModel-100/'.format(eps)
    # os.makedirs(os.path.dirname(dirname), exist_ok=True)
    model.postModels['U_of_Y'].NN.compile()

    model.postModels['U_of_Y'].NN.save(dirname+'/U_of_Y_weights.model')
    model.postModels['U_of_Y'].NN.save_weights(dirname+'/U_of_Y_weights.h5')
    print("SAVED WEIGHTS")



In [ ]:
#print(elboAll)
if trainModels:
    elboAll =  np.array(elboAll)
    np.savetxt('ELBOValsModel-{}epsi'.format(eps), elboAll)

#plt.plot(range(elboAll.shape[0]),  elboAll )
#scale = 1e3
#plt.ylim(-scale,scale)
#plt.show()


print( 'totalTimeTraining = ', totalTimeTraining)
print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

with open('TimesRecording.dat', 'w') as file:
    np.savetxt(file, totalTimeTraining)
    np.savetxt(file, totalTimeSolving)
    np.savetxt(file, totalTimeSolving)


In [ ]:
dataYEpsi =tffloat(0.010)

from matplotlib import pyplot

dirname = '100datasetmissingObsTest2/'
os.makedirs(os.path.dirname(dirname), exist_ok=True)


def addIndtNoise(u):
    return u + tf.random.normal(stddev=dataYEpsi, shape=u.shape, dtype=floatf)

def plotTrueSlnSamplesPost(samplesK, samplesF, samplesBcl, samplesBcr, SAVE=None):
    
    fig, ax = plt.subplots(nrows=2, ncols=4, sharex=False,
                                        figsize=(20, 8))
    
    u_All = tf.constant(0., shape=[1, model.dimY ], dtype=floatf)
    color = pyplot.cm.turbo(np.linspace(0, 1,samplesK.shape[0]))
    color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']
    for i in range(samplesK.shape[0]):
        
        model.pdeState.forcing =  tf.reshape(samplesF[i,:], [-1]) 
        model.pdeState.kappa   =  tf.reshape(samplesK[i,:], [-1]) 
        model.pdeState.bdLeft  =  tf.reshape(samplesBcl[i,:], []) 
        model.pdeState.bdRight =  tf.reshape(samplesBcr[i,:], []) 

#         u_prior   = solver.solve_tf_bfgs( )
        u_prior   = solver.solve_tf_linear(  )
        u_g, x_g  = observationMap(u_prior, xFE)
        u_gp      = u_g # tf.concat([u_g[:model.numEachSide], tf.constant([np.nan]) ,u_g[-model.numEachSide:]], 0)
#         x_gp      = x_g # tf.concat([x_g[:model.numEachSide], tf.constant([np.nan]) ,x_g[-model.numEachSide:]], 0)
        x_gp      = tf.concat([x_g[:model.numEachSide], tf.constant([np.nan]) ,x_g[-model.numEachSide:]], 0)

        u_gNoise  = addIndtNoise(u_g)
#         u_gNoisep = u_gNoise # tf.concat([u_gNoise[:model.numEachSide], tf.constant([np.nan]) ,u_gNoise[-model.numEachSide:]], 0)
        u_gNoisep = tf.concat([u_gNoise[:model.numEachSide], tf.constant([np.nan]) ,u_gNoise[-model.numEachSide:]], 0)

        #         u_gNoisep = tf.concat([u_gNoise[:model.numEachSide], tf.constant([np.nan]) ,u_gNoise[-model.numEachSide:]], 0)
#         u_gNoise = tf.gather_nd(u_gNoise, tf.where(tf.math.is_finite(u_gNoise)))
        u_All = tf.concat([u_All, u_gNoise[None,:] ], 0)

        ax[0,0].plot(x_gp,u_gNoisep, c=color[i])
        # ax[0,0].set_title(r'$y$')

        ax[0,1].plot(pdeState.xFE, u_prior, c=color[i])
        # ax[0,1].set_title(r'True $u(x)$')
        
        #ax[0,2].plot(pdeState.xFE, u_missPhys - u_prior,'--' ,c='k', alpha=0.5)
        # ax[0,2].plot(x_g, u_missPhysNoise - u_prior)
        # ax[0,2].set_ylim(-0.25, 0.25)
        # ax[0,2].set_title(r'True $\rho(u, x)$')

        kappaField = pdeState.kappa_fun(xFE)
        ax[0,3].plot( pdeState.xFE, kappaField, c=color[i])
        # ax[0,3].set_title(r'True $\kappa(x)$')

        meanU_phi, logvar_phi = model.postModels['U_of_Y'].Map([u_gNoise[None,:]])
        us                    = model.reparameterize( meanU_phi, logvar_phi ) 
        meanU, diagVarU = pdeState.gaussChebyGaussCoeffs(meanU_phi[0,:], tf.exp(logvar_phi[0,:]), xFE )
        std2U = 2.*tf.sqrt(diagVarU)
#         print('std2U = \n', std2U)

        ax[1,0].plot(pdeState.xFE, meanU, alpha=1., c=color[i])
        ax[1,0].fill_between(pdeState.xFE, meanU - std2U, meanU + std2U, alpha=0.1, color=color[i])
        
        ZCoeffReconstMean, ZCoeffReconstlogvar = model.priorModels['Z_of_UF'].Map([ meanU_phi, samplesF[i,:][None,:]])

        kappaCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)

        BLCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Bl'], axis=1)
        BLCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.priorModels['Z_of_UF'].multOutputLayout['Bl'], axis=1)

        BRCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Br'], axis=1)
        BRCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.priorModels['Z_of_UF'].multOutputLayout['Br'], axis=1)

        kappaCoeffReconstMean =  tf.reshape(kappaCoeffReconstMean,[-1])
        kappaCoeffReconstVar  =  tf.exp(tf.reshape(kappaCoeffReconstlogVar,[-1]))
        meanK, diagCovK = pdeState.gaussChebyGaussCoeffs(kappaCoeffReconstMean,kappaCoeffReconstVar, xFE)
        meanKT, varKT = pdeState.unscentedGaussTransf(meanK, diagCovK, pdeState.kappaBij, k=1)
        std2KT = 2*tf.sqrt(varKT)

        BlReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Bl'], axis=1)
        BrReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Br'], axis=1)

        ax[1,3].plot(pdeState.xFE, meanKT, c=color[i])
        ax[1,3].fill_between(pdeState.xFE, meanKT - std2KT, meanKT + std2KT, alpha=0.2, color=color[i])
        # ax[1,3].set_title(r'Reconstructed $\kappa(x)$')  

 
    plt.tight_layout()
    if SAVE is not None:
        plt.tight_layout()
        extent = ax[0, 0].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}Datay.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[0, 1].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}TrueSol.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[0, 2].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}TrueRho.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[0, 3].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}TrueKappa.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 0].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}Reconstructedy.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 1].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}ReconstructedSol.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 2].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}ReconstructedRho.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 3].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('{}ReconstructedKappa.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

        plt.show()
        plt.savefig('MissingPhysFig{}.pdf'.format(SAVE))
    plt.show()

    u_All = u_All[1:,:]
    return u_All


#model.dataY = plotTrueSlnSamplesPost(samplesKdata, samplesFdata, samplesBldata, samplesBrdata)

In [ ]:
numData = 5
solver = Solver( pdeState, xFE, isNonLinear=False )

samplesKdataPost   = model.LatentKappaPrior.sample(sample_shape=(numData))
# samplesFdataPost   = model.LatentForcingPrior.sample(sample_shape=(numData))
samplesFdataPost   = tf.reshape(tf.linspace(2., 5., 5), [-1, 1])
samplesBldataPost  = model.LatentBlPrior.sample(sample_shape=(numData))
samplesBrdataPost  = model.LatentBrPrior.sample(sample_shape=(numData))

# samplesKdataPost  = samplesKdata  [0:5,:]
# samplesFdataPost  = samplesFdata  [0:5,:]
# samplesBldataPost = samplesBldata [0:5,:]
# samplesBrdataPost = samplesBrdata [0:5,:]               

plotTrueSlnSamplesPost(samplesKdataPost, samplesFdataPost, samplesBldataPost, samplesBrdataPost, SAVE='indptPrior')

In [ ]:
numTestSamples = 100

samplesKdataTest  = model.LatentKappaPrior.sample(sample_shape=(numTestSamples))
samplesFdataTest  = model.LatentForcingPrior.sample(sample_shape=(numTestSamples))
# samplesFdataTest  = tf.reshape(tf.linspace(2., 5., numTestSamples), [-1, 1])
samplesBldataTest  = model.LatentBlPrior.sample(sample_shape=(numTestSamples))
samplesBrdataTest  = model.LatentBrPrior.sample(sample_shape=(numTestSamples))

NSE_u = 0.
NSE_k = 0.

for i in range (numTestSamples):

        model.pdeState.forcing =  tf.reshape(samplesFdataTest[i,:], [-1]) 
        model.pdeState.kappa   =  tf.reshape(samplesKdataTest[i,:], [-1]) 
        model.pdeState.bdLeft  =  tf.reshape(samplesBldataTest[i,:], []) 
        model.pdeState.bdRight =  tf.reshape(samplesBrdataTest[i,:], []) 

#         u_prior   = solver.solve_tf_bfgs( )
        u_prior   = solver.solve_tf_linear(  )
        u_g, x_g  = observationMap(u_prior, xFE)
        u_gp      = u_g # tf.concat([u_g[:model.numEachSide], tf.constant([np.nan]) ,u_g[-model.numEachSide:]], 0)

        u_gNoise  = addIndtNoise(u_g)

        kappaField = pdeState.kappa_fun(xFE)

        meanU_phi, logvar_phi = model.postModels['U_of_Y'].Map([u_gNoise[None,:]])
        us                    = model.reparameterize( meanU_phi, logvar_phi ) 
        meanU, diagVarU = pdeState.gaussChebyGaussCoeffs(meanU_phi[0,:], tf.exp(logvar_phi[0,:]), xFE )
        std2U = 2.*tf.sqrt(diagVarU)

        NSE_u += tf.linalg.norm(u_prior - meanU)**2/tf.linalg.norm(u_prior)**2
        numWithinU = tf.reduce_sum(tf.where(tf.math.logical_and(u_prior > meanU-std2U, u_prior < meanU+std2U), 1, 0))
        
        ZCoeffReconstMean, ZCoeffReconstlogvar = model.priorModels['Z_of_UF'].Map([ meanU_phi, samplesFdataTest[i,:][None,:]])

        kappaCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)

        kappaCoeffReconstMean =  tf.reshape(kappaCoeffReconstMean,[-1])
        kappaCoeffReconstVar  =  tf.exp(tf.reshape(kappaCoeffReconstlogVar,[-1]))
        meanK, diagCovK = pdeState.gaussChebyGaussCoeffs(kappaCoeffReconstMean,kappaCoeffReconstVar, xFE)
        meanKT, varKT = pdeState.unscentedGaussTransf(meanK, diagCovK, pdeState.kappaBij, k=1)
        std2KT = 2*tf.sqrt(varKT)

        NSE_k += tf.linalg.norm( meanKT - kappaField ) ** 2. /  tf.linalg.norm(meanKT) ** 2.
        numWithinK = tf.reduce_sum(tf.where(tf.math.logical_and(kappaField > meanKT-std2KT, kappaField < meanKT+std2KT), 1, 0))

MNSE_u = NSE_u / numTestSamples
MNSE_k = NSE_k / numTestSamples

fracWithinU = numWithinU/ (numTestSamples*u_prior.shape[0])
fracWithinK = numWithinK/ (numTestSamples*kappaField.shape[0])

print('MNSE_u = ', MNSE_u)
print('MNSE_k = ', MNSE_k)

print('fracWithinU = ', fracWithinU)
print('fracWithinK = ', fracWithinK)